The script previously used to clean the survey forms had the incorrect radius for the snow tube. Will need to recalculate the SWE and density with the correct radius of 1.89 in the formula

In [33]:
import pandas as pd
import os
import numpy as np

In [34]:
# set the in and out directory
in_dir = r"Y:\Englishman\2021\Field_Survey_data\processing\wrong_cutter_size\all"
out_dir = (r"Y:\Englishman\2021\Field_Survey_data\published")
os.chdir(in_dir)

# Set the watershed, and filename to read
wshed = "EGM"
phase = "P03"
filename = wshed + "_" + phase + ".csv"


# read the csv
df = pd.read_csv(filename)
print(len(df))
display(df.head())

373


,OBJECTID,ACO_Flight_Number,Plot_ID,SWE,Study_Area,cardinal,core,density,depth,distance,easting,height_ellipsoid_,latitude,longitude,mass,multi_core,northing,notes,plot_datetime,plot_type,plug,canopy_cover,canopy_density,canopy_height,slope,elevation,aspect
0,1,3,E3F,98.163593,Englishman,Centre,245.0,0.397423,247,NaN,386896.521,NaN,49.203193,-124.552697,1360.0,no,5451204.528,Used same tare as w3f,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,13.670105,1242.030029,83.636284
1,2,3,E3F,NaN,Englishman,Centre,NaN,NaN,201,NaN,386896.521,NaN,49.203193,-124.552697,NaN,NaN,5451204.528,NaN,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,13.670105,1242.030029,83.636284
2,3,3,E2A,NaN,Englishman,Centre,NaN,NaN,249,NaN,385876.355,NaN,49.208402,-124.566867,NaN,NaN,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,13.682495,1454.859985,77.886871
3,4,3,E2A,96.720011,Englishman,Centre,218.0,0.394776,245,NaN,385876.355,NaN,49.208402,-124.566867,1340.0,no,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,13.682495,1454.859985,77.886871
4,5,3,S4CC,NaN,Englishman,Centre,NaN,NaN,128,NaN,387179.468,NaN,49.206275,-124.548909,NaN,NaN,5451541.403,NaN,4/23/2021 12:58:00,Cardinal 10 m,0,NaN,NaN,NaN,20.065092,1262.310059,177.242859


Will change the column names of SWE and density to swe_189, density_189 so data users know problem has been ammended.

In [35]:
display(df.dtypes)
pd.set_option("display.max_columns", None)

OBJECTID               int64
ACO_Flight_Number      int64
Plot_ID               object
SWE                  float64
Study_Area            object
cardinal              object
core                 float64
density              float64
depth                  int64
distance             float64
easting              float64
height_ellipsoid_    float64
latitude             float64
longitude            float64
mass                 float64
multi_core            object
northing             float64
notes                 object
plot_datetime         object
plot_type             object
plug                   int64
canopy_cover         float64
canopy_density       float64
canopy_height        float64
slope                float64
elevation            float64
aspect               float64
dtype: object

Will need to also calculate the metrics for Englishman for aspect, canopy cover, and elevation. Will also change some of the column names to match Cruickshank

In [36]:
#delete extra elevation column
df = df.drop(columns="height_ellipsoid_")

#calculate the canopy cover 
conditions = [(df["canopy_cover"].isna()),
              (df["canopy_cover"] <= 50),
              (df["canopy_cover"] >= 50)]
values = ["open", "sparse_trees", "under_canopy"]

df["cover_type"]=np.select(conditions,values)

#calculate the aspect
conditions_asp = [
        df["aspect"] <= -1,
        df["aspect"] <= 22.5,
        df["aspect"] <= 67.5,
        df["aspect"] <= 112.5,
        df["aspect"] <= 157.5,
        df["aspect"] <= 202.5,
        df["aspect"] <= 247.5,
        df["aspect"] <= 292.5,
        df["aspect"] <= 337.5,
        df["aspect"] <= 360]

values_asp = ["Flat", "North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest", "North"]
df["aspect_type"] = np.select(conditions_asp, values_asp)
    
#round the elevation value
df["elevation_band"] = round(df["elevation"], -2)
df = df.round({'canopy_cover': 0, 'slope': 0})
    
#drop aspect column
df = df.drop(columns="aspect")


In [37]:
#ensure that "mass" column is of numeric type
df["mass"] = df["mass"].apply(pd.to_numeric, errors="coerce")

#lower case all column headers
df.columns = df.columns.str.lower()

#recalculate the SWE and density using the mass and updated tube dimesion
df["swe_189"]= df["mass"] / (3.14159265359 * (1.89 * 1.89))
df["density_189"]=df["swe_189"]/df["depth"]
display(df)

,objectid,aco_flight_number,plot_id,swe,study_area,cardinal,core,density,depth,distance,easting,latitude,longitude,mass,multi_core,northing,notes,plot_datetime,plot_type,plug,canopy_cover,canopy_density,canopy_height,slope,elevation,cover_type,aspect_type,elevation_band,swe_189,density_189
0,1,3,E3F,98.163593,Englishman,Centre,245.0,0.397423,247,NaN,386896.521,49.203193,-124.552697,1360.0,no,5451204.528,Used same tare as w3f,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,14.0,1242.030029,under_canopy,East,1200.0,121.189621,0.490646
1,2,3,E3F,NaN,Englishman,Centre,NaN,NaN,201,NaN,386896.521,49.203193,-124.552697,NaN,NaN,5451204.528,NaN,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,14.0,1242.030029,under_canopy,East,1200.0,NaN,NaN
2,3,3,E2A,NaN,Englishman,Centre,NaN,NaN,249,NaN,385876.355,49.208402,-124.566867,NaN,NaN,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,14.0,1454.859985,open,East,1500.0,NaN,NaN
3,4,3,E2A,96.720011,Englishman,Centre,218.0,0.394776,245,NaN,385876.355,49.208402,-124.566867,1340.0,no,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,14.0,1454.859985,open,East,1500.0,119.407421,0.487377
4,5,3,S4CC,NaN,Englishman,Centre,NaN,NaN,128,NaN,387179.468,49.206275,-124.548909,NaN,NaN,5451541.403,NaN,4/23/2021 12:58:00,Cardinal 10 m,0,NaN,NaN,NaN,20.0,1262.310059,open,South,1300.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,369,3,N2F,NaN,Englishman,SE,NaN,NaN,244,10.0,386566.412,49.199162,-124.557102,NaN,NaN,5450763.019,Rock,4/23/2021 10:27:00,NaN,0,NaN,NaN,NaN,7.0,1332.550049,open,Northwest,1300.0,NaN,NaN
369,370,3,N2F,85.893144,Englishman,E,180.0,0.293151,293,10.0,386566.412,49.199162,-124.557102,1190.0,no,5450763.019,Air pockets good core,4/23/2021 10:27:00,NaN,2,NaN,NaN,NaN,7.0,1332.550049,open,Northwest,1300.0,106.040918,0.361914
370,371,3,W3F,NaN,Englishman,NaN,NaN,NaN,181,7.5,386818.185,49.203339,-124.553777,NaN,NaN,5451222.384,NaN,4/23/2021 11:19:00,NaN,0,NaN,NaN,NaN,19.0,1244.339966,open,Northwest,1200.0,NaN,NaN
371,372,3,W3F,71.457321,Englishman,Centre,179.0,0.366448,195,NaN,386818.185,49.203339,-124.553777,990.0,no,5451222.384,NaN,4/23/2021 11:19:00,NaN,0,NaN,NaN,NaN,19.0,1244.339966,open,Northwest,1200.0,88.218915,0.452405


In [38]:
#comment out below if not needed, drop extra columns which are 0:3
df.drop(df.columns[0], axis =1, inplace = True)

#drop old swe and density columns
df.drop(["swe", "density"], axis=1, inplace = True)
display(df)

col_order = ["aco_flight_number", "plot_datetime", "study_area", "plot_id", "plot_type", "cardinal", "distance", "multi_core", "depth", "core", "plug", "mass", "notes", "elevation", "longitude", "latitude", "easting", "northing", "canopy_cover", "canopy_density", "canopy_height", "slope", "aspect_type", "cover_type", "elevation", "swe_189", "density_189"]
df = df[col_order]
display(df)




,aco_flight_number,plot_id,study_area,cardinal,core,depth,distance,easting,latitude,longitude,mass,multi_core,northing,notes,plot_datetime,plot_type,plug,canopy_cover,canopy_density,canopy_height,slope,elevation,cover_type,aspect_type,elevation_band,swe_189,density_189
0,3,E3F,Englishman,Centre,245.0,247,NaN,386896.521,49.203193,-124.552697,1360.0,no,5451204.528,Used same tare as w3f,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,14.0,1242.030029,under_canopy,East,1200.0,121.189621,0.490646
1,3,E3F,Englishman,Centre,NaN,201,NaN,386896.521,49.203193,-124.552697,NaN,NaN,5451204.528,NaN,4/23/2021 12:02:00,Cardinal 10 m,0,100.0,73.699997,11.0,14.0,1242.030029,under_canopy,East,1200.0,NaN,NaN
2,3,E2A,Englishman,Centre,NaN,249,NaN,385876.355,49.208402,-124.566867,NaN,NaN,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,14.0,1454.859985,open,East,1500.0,NaN,NaN
3,3,E2A,Englishman,Centre,218.0,245,NaN,385876.355,49.208402,-124.566867,1340.0,no,5451804.842,NaN,4/22/2021 7:08:00,Cardinal 10 m,0,NaN,NaN,NaN,14.0,1454.859985,open,East,1500.0,119.407421,0.487377
4,3,S4CC,Englishman,Centre,NaN,128,NaN,387179.468,49.206275,-124.548909,NaN,NaN,5451541.403,NaN,4/23/2021 12:58:00,Cardinal 10 m,0,NaN,NaN,NaN,20.0,1262.310059,open,South,1300.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,3,N2F,Englishman,SE,NaN,244,10.0,386566.412,49.199162,-124.557102,NaN,NaN,5450763.019,Rock,4/23/2021 10:27:00,NaN,0,NaN,NaN,NaN,7.0,1332.550049,open,Northwest,1300.0,NaN,NaN
369,3,N2F,Englishman,E,180.0,293,10.0,386566.412,49.199162,-124.557102,1190.0,no,5450763.019,Air pockets good core,4/23/2021 10:27:00,NaN,2,NaN,NaN,NaN,7.0,1332.550049,open,Northwest,1300.0,106.040918,0.361914
370,3,W3F,Englishman,NaN,NaN,181,7.5,386818.185,49.203339,-124.553777,NaN,NaN,5451222.384,NaN,4/23/2021 11:19:00,NaN,0,NaN,NaN,NaN,19.0,1244.339966,open,Northwest,1200.0,NaN,NaN
371,3,W3F,Englishman,Centre,179.0,195,NaN,386818.185,49.203339,-124.553777,990.0,no,5451222.384,NaN,4/23/2021 11:19:00,NaN,0,NaN,NaN,NaN,19.0,1244.339966,open,Northwest,1200.0,88.218915,0.452405


,aco_flight_number,plot_datetime,study_area,plot_id,plot_type,cardinal,distance,multi_core,depth,core,plug,mass,notes,elevation,longitude,latitude,easting,northing,canopy_cover,canopy_density,canopy_height,slope,aspect_type,cover_type,elevation,swe_189,density_189
0,3,4/23/2021 12:02:00,Englishman,E3F,Cardinal 10 m,Centre,NaN,no,247,245.0,0,1360.0,Used same tare as w3f,1242.030029,-124.552697,49.203193,386896.521,5451204.528,100.0,73.699997,11.0,14.0,East,under_canopy,1242.030029,121.189621,0.490646
1,3,4/23/2021 12:02:00,Englishman,E3F,Cardinal 10 m,Centre,NaN,NaN,201,NaN,0,NaN,NaN,1242.030029,-124.552697,49.203193,386896.521,5451204.528,100.0,73.699997,11.0,14.0,East,under_canopy,1242.030029,NaN,NaN
2,3,4/22/2021 7:08:00,Englishman,E2A,Cardinal 10 m,Centre,NaN,NaN,249,NaN,0,NaN,NaN,1454.859985,-124.566867,49.208402,385876.355,5451804.842,NaN,NaN,NaN,14.0,East,open,1454.859985,NaN,NaN
3,3,4/22/2021 7:08:00,Englishman,E2A,Cardinal 10 m,Centre,NaN,no,245,218.0,0,1340.0,NaN,1454.859985,-124.566867,49.208402,385876.355,5451804.842,NaN,NaN,NaN,14.0,East,open,1454.859985,119.407421,0.487377
4,3,4/23/2021 12:58:00,Englishman,S4CC,Cardinal 10 m,Centre,NaN,NaN,128,NaN,0,NaN,NaN,1262.310059,-124.548909,49.206275,387179.468,5451541.403,NaN,NaN,NaN,20.0,South,open,1262.310059,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,3,4/23/2021 10:27:00,Englishman,N2F,NaN,SE,10.0,NaN,244,NaN,0,NaN,Rock,1332.550049,-124.557102,49.199162,386566.412,5450763.019,NaN,NaN,NaN,7.0,Northwest,open,1332.550049,NaN,NaN
369,3,4/23/2021 10:27:00,Englishman,N2F,NaN,E,10.0,no,293,180.0,2,1190.0,Air pockets good core,1332.550049,-124.557102,49.199162,386566.412,5450763.019,NaN,NaN,NaN,7.0,Northwest,open,1332.550049,106.040918,0.361914
370,3,4/23/2021 11:19:00,Englishman,W3F,NaN,NaN,7.5,NaN,181,NaN,0,NaN,NaN,1244.339966,-124.553777,49.203339,386818.185,5451222.384,NaN,NaN,NaN,19.0,Northwest,open,1244.339966,NaN,NaN
371,3,4/23/2021 11:19:00,Englishman,W3F,NaN,Centre,NaN,no,195,179.0,0,990.0,NaN,1244.339966,-124.553777,49.203339,386818.185,5451222.384,NaN,NaN,NaN,19.0,Northwest,open,1244.339966,88.218915,0.452405


In [39]:
#drop duplicates in all columns except notes
duplicates = col_order
duplicates.remove("notes")
display(duplicates)
df.drop_duplicates(subset = duplicates)
display(len(df))

['aco_flight_number',
 'plot_datetime',
 'study_area',
 'plot_id',
 'plot_type',
 'cardinal',
 'distance',
 'multi_core',
 'depth',
 'core',
 'plug',
 'mass',
 'elevation',
 'longitude',
 'latitude',
 'easting',
 'northing',
 'canopy_cover',
 'canopy_density',
 'canopy_height',
 'slope',
 'aspect_type',
 'cover_type',
 'elevation',
 'swe_189',
 'density_189']

373

In [40]:
#export corrected version to published folder
os.chdir(out_dir)
df.to_csv(filename, index=False)